In [1]:
import os
import gradio as gr
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image

# 載入模型


In [2]:
# 定義 CNN 模型結構
class CNN(nn.Module):
    def __init__(self): # 定義初始化方法
        super(CNN, self).__init__() # 繼承父類別的初始化方法
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # 第一個卷積層
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # 第二個卷積層
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # 第三個卷積層
        self.pool = nn.MaxPool2d(2, 2)                            # 池化層
        self.adap_pool = nn.AdaptiveAvgPool2d((1, 1))             # 自適應池化層
        self.fc1 = nn.Linear(128, 512)                            # 全連接層
        self.fc2 = nn.Linear(512, 2)                              # 輸出層

    # 定義向前傳播函數，x為輸入的影像張量
    def forward(self, x):
        x = F.relu(self.conv1(x)) # 使用ReLU激活函數
        x = self.pool(x)         # 池化
        x = F.relu(self.conv2(x)) # 使用ReLU激活函數
        x = self.pool(x)        # 池化
        x = F.relu(self.conv3(x)) # 使用ReLU激活函數
        x = self.pool(x)       # 池化
        x = self.adap_pool(x)  # 自適應池化層將任何尺寸的特徵圖調整為1x1
        x = x.view(-1, 128)    # 攤平特徵圖
        x = F.relu(self.fc1(x)) # 使用ReLU激活函數
        x = self.fc2(x)        # 輸出層
        return x              # 返回輸出

# 載入模型
model = CNN()
model.load_state_dict(torch.load('pneumonia_model.pth', map_location=torch.device('cpu')))
model.eval()

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (adap_pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc1): Linear(in_features=128, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

# 定義影像處理轉換


In [3]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229]),
])


# 定義預測函數

In [4]:
def predict(image):
    image = Image.fromarray(image.astype('uint8'), 'RGB')
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        outputs = model(image)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        top_p, top_class = probabilities.topk(1, dim=1)
        predicted_class = top_class[0][0].item()
        confidence = top_p[0][0].item()

    label = '正常' if predicted_class == 0 else '肺炎'
    return f'{label}, 信心度: {confidence*100:.2f}%'


# 啟動界面


In [5]:
# 指定測試數據集中的 NORMAL 和 PNEUMONIA 資料夾路徑
test_normal_path = 'chest_xray/test/NORMAL'
test_pneumonia_path = 'chest_xray/test/PNEUMONIA'

# 選擇每個資料夾中的前十個圖片作為示例
example_normals = [os.path.join(test_normal_path, fname) for fname in os.listdir(test_normal_path)[:10]]
example_pneumonias = [os.path.join(test_pneumonia_path, fname) for fname in os.listdir(test_pneumonia_path)[:10]]

# 確保所有選定的圖片路徑都存在
for path in example_normals + example_pneumonias:
    if not os.path.exists(path):
        raise ValueError(f"檔案不存在：{path}")

# 將所有示例圖片的路徑合併到一個列表中
all_examples = example_normals + example_pneumonias

# Gradio 界面
iface = gr.Interface(fn=predict, 
                     inputs=gr.inputs.Image(shape=(256, 256)), 
                     outputs='text', 
                     title='肺炎檢測模型',
                     examples=all_examples)
iface.launch(share=True)

C:\Users\Alan\AppData\Local\Temp\ipykernel_18364\1298634196.py:19: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Image(shape=(256, 256)),
C:\Users\Alan\AppData\Local\Temp\ipykernel_18364\1298634196.py:19: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Image(shape=(256, 256)),


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://ca00c46730e9b04a53.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
